## 🛠️ **Primer prototipo o pruebas con datos**

En esta etapa necesitamos:

1. **Cargar** una oración de prueba.
2. **Definir** manualmente los parámetros del HMM (π, A, B).
3. **Preparar estructuras** para procesarlas en código.

### 📘 TEORÍA DE HMM (Modelo Oculto de Markov)

Según Rabiner:

- Un HMM es un modelo estadístico probabilístico en el que se asume que el sistema que se modela es un proceso de Markov con estados ocultos. donde:
    - El estado actual influye en los estados futuros, pero los estados en sí no son directamente observables.
    - Hay una secuencia de observaciones conocidas, lo que observamos son **emisiones** o **observaciones** que dependen del estado oculto actual.
    - Hay una secuencia de **estados ocultos** desconocidos que queremos inferir.
- Se compone de:
    - **Conjunto de estados ocultos (S):** Un número finito de estados que no son directamente observables. En el contexto del POS-tagging, estos estados representarían las posibles etiquetas gramaticales (sustantivo, verbo, adjetivo, etc.).
    - **Conjunto de observaciones (V):** Un número finito de símbolos de observación que son las salidas del sistema que podemos observar. En el POS-tagging, estas serían las palabras del texto que queremos etiquetar.
- Se caracteriza por 3 conjuntos de parámetros:
    - **Matriz de probabilidades de transición de estados (A):** Define la probabilidad de transición entre los estados ocultos. $a_{ij} = P(q_{t+1} = S_j | q_t = S_i)$ es la probabilidad de estar en el estado $S_i$  en el tiempo $t$ y pasar al estado $S_j$ en el tiempo $t+1$.  ***(probabilidad de pasar de una etiqueta a otra)***
    - **Matriz de probabilidades de emisión (B):** Define la probabilidad de observar un símbolo particular dado un estado oculto. $b_j(k) = P(O_t = v_k | q_t = S_j)$ es la probabilidad de observar el símbolo $v_k$ cuando el estado oculto en el tiempo $t$ es $S_j$. ***(probabilidad de que una palabra sea generada por una etiqueta.)***
    - **Vector de probabilidades iniciales de estado (π):** Define la probabilidad de estar en un estado particular en el tiempo inicial ($t=1$).
        
         $\pi_i = P(q_1 = S_i)$ es la probabilidad de que el primer estado oculto sea $S_i$. ***(probabilidad de comenzar en cada etiqueta.)***
        

**HMM de Primer Orden**

Un HMM de primer orden se caracteriza por la **propiedad de Markov de primer orden**, que establece que la probabilidad del estado actual solo depende del estado inmediatamente anterior. Matemáticamente, esto se expresa como:

$P(q_t | q_{t-1}, q_{t-2}, ..., q_1) = P(q_t | q_{t-1})$

De manera similar, la probabilidad de una observación en un momento dado solo depende del estado oculto en ese mismo momento:

$P(O_t | q_1, ..., q_t, O_1, ..., O_{t-1}) = P(O_t | q_t)$

**HMM para POS-Tagging**

En el contexto del POS-tagging, podemos mapear los componentes de un HMM de la siguiente manera:

- **Estados ocultos (S):** El conjunto de posibles etiquetas gramaticales (e.g., {NOUN, VERB, ADJ, DET}).
- **Observaciones (V):** El vocabulario del idioma (el conjunto de todas las palabras posibles).
- **Probabilidades de transición (A):** La probabilidad de que una etiqueta gramatical siga a otra (e.g., la probabilidad de que un sustantivo siga a un determinante). Estas probabilidades se pueden estimar a partir de un corpus de texto etiquetado.
- **Probabilidades de emisión (B):** La probabilidad de que una palabra particular sea generada por una etiqueta gramatical específica (e.g., la probabilidad de que la palabra "gato" tenga la etiqueta NOUN). Estas probabilidades también se pueden estimar a partir de un corpus de texto etiquetado.
- **Probabilidades iniciales (π):** La probabilidad de que una etiqueta gramatical sea la primera etiqueta en una oración (e.g., la probabilidad de que la primera palabra de una oración sea un determinante o un sustantivo).

### 🧪 PRUEBA PRÁCTICA: Implementación básica del corpus "Time flies like an arrow"

✅ **¿Qué lograramos con esto?**

- Armas el HMM desde cero.
- El código es totalmente controlado (no usa librerías externas de NLP).
- Prepara la base para aplicar **el algoritmo de Viterbi en el siguiente paso**.

In [1]:
import numpy as np
import pandas as pd

# Oración de prueba
sentence = ["Time", "flies", "like", "an", "arrow"]

# Estados posibles
states = ["NOUN", "VERB", "DET", "PREP"]
observations = sentence

# Índices
state_idx = {s: i for i, s in enumerate(states)}
obs_idx = {w: i for i, w in enumerate(observations)}

### 1. 🔵 π – Probabilidades iniciales

In [2]:
# π: Probabilidades iniciales
pi = [0.4, 0.3, 0.2, 0.1]  # [NOUN, VERB, DET, PREP]

### 📌 ¿Qué representa?

Es la **probabilidad de comenzar** una oración con cada una de las etiquetas posibles. Por ejemplo:

- ¿Qué tan probable es que la primera palabra de una oración sea un **sustantivo** (`NOUN`)? Bastante común → 0.4
- ¿Un verbo al inicio? Posible pero no tanto → 0.3
- ¿Un determinante (`DET`)? Como "The", "An" → 0.2
- ¿Una preposición (`PREP`)? Muy raro → 0.1

### 📋 Criterio usado:

- Uso **intuitivo / heurístico** basado en cómo suelen comenzar las oraciones en inglés.
- No usamos un corpus grande para entrenarlo (aún), así que lo **asignamos a mano de forma razonada**.

### 2. 🔁 `A` – Matriz de transición de estados

In [3]:
# A: Matriz de transición
A = np.array([
    [0.1, 0.6, 0.2, 0.1],  # desde NOUN
    [0.3, 0.1, 0.1, 0.5],  # desde VERB
    [0.7, 0.1, 0.1, 0.1],  # desde DET
    [0.6, 0.2, 0.1, 0.1],  # desde PREP
])

### 📌 ¿Qué representa?

Cada fila dice: "si estoy en la etiqueta X, ¿a qué etiqueta paso después con qué probabilidad?"

### Ejemplos:

- Desde `DET`, la probabilidad de pasar a `NOUN` es alta → 0.7 (como en "an arrow").
- Desde `NOUN`, es común que siga un verbo → 0.6
- Desde `VERB`, es común que venga una preposición → 0.5 (como en "like an arrow").

### 📋 Criterio usado:

- Heurística inspirada en reglas gramaticales típicas del inglés.
- Usamos **conocimiento lingüístico general** (y un poco de lógica narrativa) para simular lo que haría un corpus entrenado.

### 3. 🟣 `B` – Matriz de emisión

In [4]:
# B: Matriz de emisión
B = np.array([
    [0.3, 0.2, 0.1, 0.0, 0.4],  # NOUN
    [0.1, 0.6, 0.2, 0.0, 0.1],  # VERB
    [0.0, 0.0, 0.0, 1.0, 0.0],  # DET
    [0.0, 0.0, 0.9, 0.1, 0.0],  # PREP
])

### 📌 ¿Qué representa?

Cada fila te dice: "¿cuál es la probabilidad de que la palabra *X* haya sido generada por esta etiqueta?"

### Ejemplos:

- `"flies"` → puede ser `VERB` (0.6) o `NOUN` (0.2) → ¡ambigüedad clásica!
- `"like"` → puede ser `PREP` (0.9) o `VERB` (0.2)
- `"an"` → clarísimo que es `DET` (1.0)
- `"arrow"` → muy probablemente un `NOUN` (0.4)

### 📋 Criterio usado:

- Analizamos semántica y **ambigüedad léxica** de cada palabra.
- Los valores se asignan **a mano**, con la idea de **reflejar ambigüedad realista**.

In [5]:
# Mostrar matrices
print("Estados:", states)
print("Observaciones:", observations)
print("π:", pi)
print("A:\n", pd.DataFrame(A, index=states, columns=states))
print("B:\n", pd.DataFrame(B, index=states, columns=observations))

Estados: ['NOUN', 'VERB', 'DET', 'PREP']
Observaciones: ['Time', 'flies', 'like', 'an', 'arrow']
π: [0.4, 0.3, 0.2, 0.1]
A:
       NOUN  VERB  DET  PREP
NOUN   0.1   0.6  0.2   0.1
VERB   0.3   0.1  0.1   0.5
DET    0.7   0.1  0.1   0.1
PREP   0.6   0.2  0.1   0.1
B:
       Time  flies  like   an  arrow
NOUN   0.3    0.2   0.1  0.0    0.4
VERB   0.1    0.6   0.2  0.0    0.1
DET    0.0    0.0   0.0  1.0    0.0
PREP   0.0    0.0   0.9  0.1    0.0


## ¿Por qué no usamos datos reales?

Porque estamos en una **fase inicial/prototipo** donde queremos entender bien cómo funciona el modelo. Estos valores nos permiten:

- Testear el algoritmo de Viterbi sin necesidad de entrenamiento.
- Ver cómo cambia la secuencia etiquetada si alteramos las probabilidades